<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/Violence_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
from IPython.display import clear_output 

#Datasets

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset
!unzip \*.zip && rm *.zip
!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git
clear_output()

In [3]:
os.makedirs('/content/NonViolence/', exist_ok=True)
os.makedirs('/content/Violence/', exist_ok=True)

nv = '/content/Real Life Violence Dataset/NonViolence/'
v = '/content/Real Life Violence Dataset/Violence/'

for V in [v, nv]:
  for i in os.listdir(V):
    R = V+i
    os.replace(R, R.replace('Real Life Violence Dataset/', ''))

In [4]:
path = '/content/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/'
choices = ['/content/NonViolence/', '/content/NonViolence/', '/content/Violence/', '/content/Violence/']

for id, V in enumerate(['non-violent/cam1/', 'non-violent/cam2/', 'violent/cam1/', 'violent/cam2/']):
  for i in os.listdir(path+V):
    os.replace(path+V+i, choices[id]+i)

In [5]:
os.makedirs('/content/data/', exist_ok=True)
os.replace('/content/NonViolence/', '/content/data/NonViolence/')
os.replace('/content/Violence/', '/content/data/Violence/')

#2

In [ ]:
!pip uninstall emoji
!pip install emoji==1.7.0
!pip install text2emotion

import text2emotion as te

clear_output()

In [ ]:
sample = "I was asked to sign a third party contract a week out from stay"
te.get_emotion(sample)

{'Happy': 0.0, 'Angry': 0.25, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 0.75}

https://www.kaggle.com/datasets/adiamaan/movie-subtitle-dataset/code

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d adiamaan/movie-subtitle-dataset
!unzip \*.zip && rm *.zip
clear_output()

In [ ]:
import pandas as pd
sub = pd.read_csv('/content/movies_subtitles.csv')
meta = pd.read_csv('/content/movies_meta.csv')

In [ ]:

abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def convert_abb(x):
    if type(x) != str:
      return str(x)
    word_list = x.split()
    r_string = []
    for word in word_list:
        r_string.append(abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word)
    return ' '.join(r_string)


In [ ]:
sub['text'] = sub.text.apply(convert_abb)

# # Remove emojis
# train_df['clean_text'] = train_df.clean_text.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
# test_df['clean_text'] = test_df.clean_text.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [ ]:
# sub['text'] = sub.text.apply(convert_abb)

ss = sub.text.apply(lambda x: te.get_emotion(x))

In [ ]:
import json
with open('/content/drive/MyDrive/forage/data.json', 'w', encoding='utf-8') as f:
    json.dump(ss, f, ensure_ascii=False, indent=4)

In [ ]:
ll

In [ ]:
import json
with open('/content/drive/MyDrive/forage/data.json', 'w', encoding='utf-8') as f:
    json.dump(ss, f, ensure_ascii=False, indent=4)

#Faces Extract

In [13]:
# #import shutil
# import os
# import cv2
# import random
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

!mkdir frame
!mkdir faces

# os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
# !kaggle datasets download -d erikvdven/tiktok-trending-december-2020
# !unzip \*.zip && rm *.zip
# clear_output()

# paths=[]
# for dirname, _, filenames in os.walk('/content/tiktok-trending-december-2020/videos'):
#     for filename in filenames:
#         paths+=[(os.path.join(dirname, filename))]
# paths=sorted(paths)

# d_frame='/content/frame/'
# d_faces='/content/faces/'

def video_2_frames(video_file, image_dir=d_frame, image_file='img_%s.png'):
    i = 0
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        cv2.imwrite(image_dir+image_file % str(i).zfill(6), frame) 
        i += 1
    cap.release()

In [14]:
video_2_frames('/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_1.mp4')

frame_paths=[]
for file in os.listdir(d_frame):
    frame_paths+=[os.path.join(d_frame,file)]
frame_paths=sorted(frame_paths)
print(frame_paths[0:5])

['/content/frame/img_000000.png', '/content/frame/img_000001.png', '/content/frame/img_000002.png', '/content/frame/img_000003.png', '/content/frame/img_000004.png']


In [ ]:
!wget https://raw.githubusercontent.com/kipr/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

path_cas='/kaggle/inpu/content/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(path_cas)

In [49]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml
# !wget --no-check-certificate \
#     https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/utils/common.py \
#     -O common.py

import cv2 
# import common #some useful opencv functions
import numpy as np

counter = 0
for img_path in frame_paths:
  image = cv2.imread(img_path)
  face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
  faces = face_cascade.detectMultiScale(image, 1.3, 5)
  print(faces)
  for (x, y, w, h) in faces:
      counter += 1
      cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
      # face = image[y:y + h, x:x + w]
      offset = 10
      face_section = image[y-offset:y+h+offset,x-offset:x+w+offset]
      face_section = cv2.resize(face_section,(100,100))

      print(face_section)
      cv2.imwrite(f'/content/face{counter}.jpg', face_section)
  # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

--2023-05-21 21:14:48--  https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.03s   

2023-05-21 21:14:49 (27.6 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()


In [22]:
frame_paths=[]
for file in os.listdir(d_frame):
    frame_paths+=[os.path.join(d_frame,file)]
frame_paths=sorted(frame_paths)
print(frame_paths[0:5])

['/content/frame/img_000000.png', '/content/frame/img_000001.png', '/content/frame/img_000002.png', '/content/frame/img_000003.png', '/content/frame/img_000004.png']


In [23]:
import cv2
#Init Camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

while True:
    ret,frame = cap.read()
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret==False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    print(faces)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.rectangle(gray_frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Video Frame",frame)
    cv2.imshow("Gray Frame" ,gray_frame)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [24]:
import cv2
import numpy as np

#Init Camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip = 0
face_data = []
dataset_path = './image_data/'
file_name = input("Enter the name of the person : ")
while True:
    ret,frame = cap.read()

    if ret==False:
        continue


    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if len(faces)==0:
        continue

    faces = sorted(faces,key=lambda f:f[2]*f[3])

    # Pick the last face (because it is the largest face acc to area(f[2]*f[3]))
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)



    #Extract (Crop out the required face) : Region of Interest
    offset = 10
    face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
    face_section = cv2.resize(face_section,(100,100))

    skip += 1
    if skip%10==0:
        face_data.append(face_section)
        print(len(face_data))


    cv2.imshow("Frame",frame)
    cv2.imshow("Face Section",face_section)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

# Convert our face list array into a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

# Save this data into file system
np.save(dataset_path+file_name+'.npy',face_data)
print("Data Successfully save at "+dataset_path+file_name+'.npy')

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: ignored

In [ ]:
import cv2
import numpy as np 
import os 

def distance(v1, v2):
    # Eucledian 
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]


#Init Camera
# cap = cv2.VideoCapture(0)

# # Face Detection
# face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

# skip = 0
# dataset_path = './image_data/'

# face_data = [] 
# labels = []

# class_id = 0 # Labels for the given file
# names = {} #Mapping btw id - name


# # Data Preparation
# for fx in os.listdir(dataset_path):
#     if fx.endswith('.npy'):
#        #Create a mapping btw class_id and name
#         names[class_id] = fx[:-4]
#         print("Loaded "+fx)
#         data_item = np.load(dataset_path+fx)
#         face_data.append(data_item)

#         #Create Labels for the class
#         target = class_id*np.ones((data_item.shape[0],))
#         class_id += 1
#         labels.append(target)

# face_dataset = np.concatenate(face_data,axis=0)
# face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

# print(face_dataset.shape)
# print(face_labels.shape)

# trainset = np.concatenate((face_dataset,face_labels),axis=1)
# print(trainset.shape)

# Testing 

while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

        #Get the face ROI
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        #Predicted Label (out)
        out = knn(trainset,face_section.flatten())

        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#ddd

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d chaditya95/epileptic-seizures-dataset
!unzip \*.zip && rm *.zip

In [29]:
data = pd.read_csv('/content/data.csv')
data

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,...,15,16,12,5,-1,-18,-37,-47,-48,2
11496,X19.V1.354,-47,-11,28,77,141,211,246,240,193,...,-65,-33,-7,14,27,48,77,117,170,1
11497,X8.V1.28,14,6,-13,-16,10,26,27,-9,4,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,X10.V1.932,-40,-25,-9,-12,-2,12,7,19,22,...,121,135,148,143,116,86,68,59,55,3


In [33]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)
dataset=pd.DataFrame(data.iloc[:,0:-1])
dataset

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,135,190,229,223,192,125,55,-9,-33,-38,...,8,-17,-15,-31,-77,-103,-127,-116,-83,-51
1,386,382,356,331,320,315,307,272,244,232,...,168,164,150,146,152,157,156,154,143,129
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,29,57,64,48,19,-12,-30,-35,-35,-36
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,10,4,2,-12,-32,-41,-65,-83,-89,-73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,20,15,16,12,5,-1,-18,-37,-47,-48
11496,-47,-11,28,77,141,211,246,240,193,136,...,-94,-65,-33,-7,14,27,48,77,117,170
11497,14,6,-13,-16,10,26,27,-9,4,14,...,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8
11498,-40,-25,-9,-12,-2,12,7,19,22,29,...,114,121,135,148,143,116,86,68,59,55


In [34]:
from sklearn.preprocessing  import StandardScaler
scaler=StandardScaler()
scaled_data=scaler.fit_transform(dataset)

In [36]:
X=scaled_data
y=data['y']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

from sklearn.neural_network import MLPClassifier
rf=MLPClassifier()
model=rf.fit(X_train,y_train)
pred=model.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix
print("Confusion Matrix:\n",confusion_matrix(y_test,pred))
print("Classification Report:\n",classification_report(y_test,pred))

Confusion Matrix:
 [[525  18   9  22   2]
 [ 14 332 195   6  31]
 [ 12 202 311   8  48]
 [  4  10  12 426 129]
 [  0  48  49 107 355]]
Classification Report:
               precision    recall  f1-score   support

           1       0.95      0.91      0.93       576
           2       0.54      0.57      0.56       578
           3       0.54      0.54      0.54       581
           4       0.75      0.73      0.74       581
           5       0.63      0.64      0.63       559

    accuracy                           0.68      2875
   macro avg       0.68      0.68      0.68      2875
weighted avg       0.68      0.68      0.68      2875



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers


hidden_units=50
learning_rate=0.001
hidden_layer_act='relu'
output_layer_act='sigmoid'
no_epochs=5


model = Sequential()
model.add(Dense(hidden_units, input_dim=178, activation='relu'))
model.add(Dense(hidden_units, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

sgd=optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=no_epochs, batch_size=1,  verbose=1)

Epoch 1/5
8625/8625 [==============================] - 18s 2ms/step - loss: -1766377.8750 - accuracy: 0.1999
Epoch 2/5
8625/8625 [==============================] - 16s 2ms/step - loss: -17664320.0000 - accuracy: 0.1999
Epoch 3/5
8625/8625 [==============================] - 17s 2ms/step - loss: -62639928.0000 - accuracy: 0.1999
Epoch 4/5
8625/8625 [==============================] - 17s 2ms/step - loss: -148446560.0000 - accuracy: 0.1999
Epoch 5/5
8625/8625 [==============================] - 17s 2ms/step - loss: -288014112.0000 - accuracy: 0.1999


In [48]:
model.evaluate(X_test, y_test)

90/90 [==============================] - 0s 2ms/step - loss: -373695648.0000 - accuracy: 0.2003


[-373695648.0, 0.20034782588481903]